<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from random import randint

# Define a simple neural network as the candidate architecture
class SimpleNN(nn.Module):
    def __init__(self, hidden_units):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, hidden_units)
        self.fc2 = nn.Linear(hidden_units, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Simulated NAS Search Space (Number of hidden units as a search space)
hidden_units_space = [64, 128, 256, 512]

# Basic reward function based on accuracy
def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# DataLoader for training and validation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example of reinforcement learning-based search (simplified)
best_accuracy = 0
best_architecture = None
best_model = None

for epoch in range(10):
    # Randomly choose an architecture from the search space
    hidden_units = hidden_units_space[randint(0, len(hidden_units_space) - 1)]
    model = SimpleNN(hidden_units=hidden_units).to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    # Train the model
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the validation set
    accuracy = evaluate_model(model, val_loader, device)
    print(f'Epoch {epoch+1}, Hidden Units: {hidden_units}, Accuracy: {accuracy:.4f}')

    # Track the best architecture based on validation accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_architecture = hidden_units

print(f'Best Architecture: {best_architecture}, Best Accuracy: {best_accuracy:.4f}')